# Install and Import Dependencies

In [1]:
!pip install google-cloud-aiplatform
!pip install --user datasets
!pip install --user google-cloud-pipeline-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import IPython
from google.cloud import aiplatform
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [2]:
import vertexai
PROJECT_ID = "YOUR_PROJECT_ID" #@param
vertexai.init(project=PROJECT_ID)

In [3]:
region = "us-central1"
REGION = "us-central1"
project_id = "YOUR_PROJECT_ID"

In [4]:
! gcloud config set project {project_id}

Updated property [core/project].


In [5]:
#Import the necessary libraries

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)
import kfp
import sys
import uuid
import json
import vertexai
import pandas as pd
from google.auth import default
from datasets import load_dataset
from google.cloud import aiplatform
from vertexai.preview.language_models import TextGenerationModel, EvaluationTextSummarizationSpec


# Prepare & Load Training Data

In [6]:
BUCKET_NAME = 'YOUR_BUCKET'
BUCKET_URI = f"gs://YOUR_BUCKET/TRAIN.jsonl"
REGION = "us-central1"

In [7]:
json_url = 'https://storage.googleapis.com/YOUR_BUCKET/TRAIN.jsonl'
df = pd.read_json(json_url, lines=True)
print (df)

                                            input_text  \
0    The BBC News website takes a look at how games...   
1    The explosion in consumer technology is to con...   
2    The proportion of surfers using Microsoft's In...   
3    'God games' in which players must control virt...   
4    Online communities set up by the UK government...   
..                                                 ...   
739  MTV has been criticised for "incessant sleaze"...   
740  Actress Anna Keaveney, who appeared in Brooksi...   
741  The permanent replacement for late DJ John Pee...   
742  Ocean's Twelve, the crime caper sequel starrin...   
743  Digital music downloads are being included in ...   

                           output_text  
0             Mobile games come of age  
1      Gadget market 'to grow in 2005'  
2    New browser wins over net surfers  
3      Games help you 'learn and play'  
4       Online commons to spark debate  
..                                 ...  
739    Campaigne

In [ ]:
print(df.shape)

(744, 2)


Fine Tune Text Bison@002 Model

In [ ]:
model_display_name = 'bbc-finetuned-model' # @param {type:"string"}
tuned_model = TextGenerationModel.from_pretrained("text-bison@002")
tuned_model.tune_model(
training_data=df,
train_steps=100,
tuning_job_location="europe-west4",
tuned_model_location="europe-west4",
)


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/273845608377/locations/europe-west4/pipelineJobs/tune-large-model-20240403173809
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/273845608377/locations/europe-west4/pipelineJobs/tune-large-model-20240403173809')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west4/pipelines/runs/tune-large-model-20240403173809?project=273845608377
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/273845608377/locations/europe-west4/pipelineJobs/tune-large-model-20240403173809 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/273845608377/locations/europe-west4/pipelineJ

# Predict with the new Fine Tuned Model

In [ ]:
response = tuned_model.predict("Summarize this text to generate a title: \n Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable it it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.")
print(response.text)

 Shrinking space on planes putting our health and safety in danger


In [ ]:
tuned_model_name = tuned_model._endpoint.gca_resource.deployed_models[0].model
tuned_model_1 = TextGenerationModel.get_tuned_model(tuned_model_name)
#TextGenerationModel.get_tuned_model("bbc-finetuned-model")
response = tuned_model_1.predict("Summarize this text to generate a title: \n Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable it it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.")
print(response.text)

 Shrinking space on planes putting our health and safety in danger


# Predict with Base Model for comparison

In [ ]:
base_model = TextGenerationModel.from_pretrained("text-bison@002")
response = base_model.predict("Summarize this text to generate a title: \n Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable it it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans.")
print(response.text)

 Shrinking Space on Planes: Putting Our Health and Safety at Risk?
